In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neuralop
import torch
from torch import nn

from denoising import (
    DatasetRegistry,
    ModelRegistry,
)
from config import (
    Environment,
    get_model_configs,
    get_datasets_configs,
)

In [3]:
print('torch ' + torch.__version__)
print('neuralop ' + neuralop.__version__)

torch 2.6.0+cu124
neuralop 1.0.2


In [4]:
# загружаем переменные окружения
env = Environment(_env_file='../env')
print(f'Weights_dir: {env.weights}')

Weights_dir: /home/d.nesterov/denoising-fno/notebooks


# Инициализация датасетов

In [5]:
dataset_registry = DatasetRegistry()
dataset_registry.load(get_datasets_configs(env.data), verbose=True)
dataset_registry

Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 6704  in dataset mri_gt_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 1676  in dataset mri_gt_val          with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])


DatasetRegistry(['mri_pm_train', 'mri_pm_test', 'mri_gt_train', 'mri_gt_val', 'mri_gt_test', 'bsd_synth_0.01_train', 'bsd_synth_0.01_test', 'sidd_train', 'sidd_test'])

# Инициализация моделей

In [6]:
model_registry = ModelRegistry()
model_registry.load(
    get_model_configs(env.weights), random_seed=42, device='cpu', verbose=True
)
model_registry

Loaded  model mri-fno-neuralop with n_parameters = 2010449
Loaded  model mri-fno-tucker   with n_parameters = 2010449
Loaded  model mri-fno-dense    with n_parameters = 4476513
Loaded  model mri-hno          with n_parameters = 1098977
Loaded  model mri-hno-optuned  with n_parameters = 10552961
Loaded  model mri-fno-optuned  with n_parameters = 4197273
Loaded  model mri-hno-gt-optuned with n_parameters = 4223585
Loaded  model mri-fno-gt-optuned with n_parameters = 212765
Loaded  model sidd-fno-run2    with n_parameters = 2011091
Loaded  model sidd-fno-run3    with n_parameters = 2011091
Loaded  model sidd-fno-run4    with n_parameters = 2028627
Loaded  model bsd-fno          with n_parameters = 2010449


ModelRegistry(['mri-fno-neuralop', 'mri-fno-tucker', 'mri-fno-dense', 'mri-hno', 'mri-hno-optuned', 'mri-fno-optuned', 'mri-hno-gt-optuned', 'mri-fno-gt-optuned', 'sidd-fno-run2', 'sidd-fno-run3', 'sidd-fno-run4', 'bsd-fno'])

# Инференс и подсчет метрик

In [8]:
from torchmetrics.image import StructuralSimilarityIndexMeasure as SSIM
from torchmetrics.image import PeakSignalNoiseRatio as PSNR

from denoising import Metrics
from config import make_metric_config

In [10]:
metrics = Metrics([
    make_metric_config('ssim', SSIM, {'data_range': 1.0}),
    make_metric_config('psnr', PSNR, {}),
])

In [11]:
print(f'Available datasets: {dataset_registry}')
print(f'Available models:   {model_registry}')
print(f'Metrics to compute: {metrics}')

Available datasets: DatasetRegistry(['mri_pm_train', 'mri_pm_test', 'mri_gt_train', 'mri_gt_val', 'mri_gt_test', 'bsd_synth_0.01_train', 'bsd_synth_0.01_test', 'sidd_train', 'sidd_test'])
Available models:   ModelRegistry(['mri-fno-neuralop', 'mri-fno-tucker', 'mri-fno-dense', 'mri-hno', 'mri-hno-optuned', 'mri-fno-optuned', 'mri-hno-gt-optuned', 'mri-fno-gt-optuned', 'sidd-fno-run2', 'sidd-fno-run3', 'sidd-fno-run4', 'bsd-fno'])
Metrics to compute: Metrics(['ssim', 'psnr'])


In [12]:
from denoising import Evaluator

In [13]:
ev = Evaluator(model_registry, dataset_registry, metrics, device='cuda:2')

In [16]:
m_mri_pm = ev.evaluate(
    [
        'mri-hno-optuned',
        'mri-fno-optuned',

        'mri-hno-gt-optuned',
        'mri-fno-gt-optuned',
        
        'mri_pm_test',
    ],
    # 'mri_gt_test',
    'mri_pm_test',
    batch_size=128,
    skip_cache=True,
)

Computing predicts for ('mri-hno-optuned', 'mri_pm_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:09<00:00,  1.70it/s]


Saved predicts for ('mri-hno-optuned', 'mri_pm_test') to cache
Computing predicts for ('mri-fno-optuned', 'mri_pm_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.89it/s]


Saved predicts for ('mri-fno-optuned', 'mri_pm_test') to cache
Computing predicts for ('mri-hno-gt-optuned', 'mri_pm_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.44it/s]


Saved predicts for ('mri-hno-gt-optuned', 'mri_pm_test') to cache
Computing predicts for ('mri-fno-gt-optuned', 'mri_pm_test') from model


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:02<00:00,  7.47it/s]


Saved predicts for ('mri-fno-gt-optuned', 'mri_pm_test') to cache


In [23]:
m_sidd = ev.evaluate(
    ['sidd-fno-run4'],
    # ['sidd-fno-run4', 'sidd-hno-run5'],
    'sidd_test',
    batch_size=32,
)
m_sidd

Retrieving predicts for ('sidd-fno-run4', 'sidd_test') from cache


{'sidd-fno-run4_ssim': tensor(0.5579)}

In [27]:
m_bsd = ev.evaluate(['bsd-fno'], 'bsd_synth_0.01_test', batch_size=128)
m_bsd

Computing predicts for ('bsd-fno', 'bsd_synth_0.01_test') from model


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.63s/it]


Saved predicts for ('bsd-fno', 'bsd_synth_0.01_test') to cache


{'bsd-fno_ssim': tensor(0.1189)}

In [15]:
m_mri

{'mri-hno-optuned_ssim': tensor(0.9240),
 'mri-hno-optuned_psnr': tensor(31.9885),
 'mri-fno-optuned_ssim': tensor(0.9170),
 'mri-fno-optuned_psnr': tensor(31.9722),
 'mri-hno-gt-optuned_ssim': tensor(0.9635),
 'mri-hno-gt-optuned_psnr': tensor(37.5799),
 'mri-fno-gt-optuned_ssim': tensor(0.9585),
 'mri-fno-gt-optuned_psnr': tensor(35.8506),
 'mri_pm_test_ssim': tensor(0.9360),
 'mri_pm_test_psnr': tensor(32.0755)}

In [17]:
m_mri_pm

{'mri-hno-optuned_ssim': tensor(0.9756),
 'mri-hno-optuned_psnr': tensor(46.2849),
 'mri-fno-optuned_ssim': tensor(0.9652),
 'mri-fno-optuned_psnr': tensor(46.0216),
 'mri-hno-gt-optuned_ssim': tensor(0.9406),
 'mri-hno-gt-optuned_psnr': tensor(34.0859),
 'mri-fno-gt-optuned_ssim': tensor(0.9452),
 'mri-fno-gt-optuned_psnr': tensor(34.2670),
 'mri_pm_test_ssim': tensor(1.),
 'mri_pm_test_psnr': tensor(inf)}

In [21]:
m_sidd

{'sidd-fno-run2_ssim': tensor(0.8911),
 'sidd-fno-run3_ssim': tensor(0.8871),
 'sidd-fno-run4_ssim': tensor(0.9105)}

In [31]:
m_bsd

{'bsd-fno_ssim': tensor(0.1189)}

# Подсчет inference time

In [7]:
import time
from tqdm import tqdm

from denoising.models.utils import count_parameters

In [10]:
@torch.no_grad()
def compute_mean_inf_time(m: nn.Module, dl: torch.utils.data.DataLoader, device: str = 'cpu') -> float:
    m = m.to(device)
    times = []
    for batch in tqdm(dl):
        b = {k: v.to(device) for k, v in batch.items()}
        s = time.perf_counter()
        _ = m(**b)
        e = time.perf_counter()
        times.append(e - s)
    m = m.to('cpu')
    return float(torch.Tensor(times).mean())

def fmt_s(s: float) -> str:
    return f'{round(s * 1000, 2)}'

In [9]:
fno = model_registry['mri-fno-gt-optuned']
hno = model_registry['mri-hno-gt-optuned']
dl = dataset_registry.make_dl('mri_pm_train', 1)

print(f'FNO params: {count_parameters(fno)}, HNO params: {count_parameters(hno)}')

FNO params: 212765, HNO params: 4223585


In [60]:
fno_t = compute_mean_inf_time(fno, dl, 'cuda:2')
fmt_s(fno_t)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8380/8380 [00:31<00:00, 264.37it/s]


0.002874373458325863

In [61]:
hno_t = compute_mean_inf_time(hno, dl, 'cuda:2')
fmt_s(hno_t)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8380/8380 [00:46<00:00, 179.48it/s]


0.004645488224923611